In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [3]:
!pip install transformers datasets sacrebleu sacremoses -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 61.7 MB/s eta 0:00:00


In [4]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00


In [5]:
import transformers
from transformers import MarianMTModel, MarianTokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict
import pandas as pd
import evaluate
import numpy as np
import torch


In [6]:
from datasets import Dataset, DatasetDict


In [7]:
path_to_train = '/content/drive/MyDrive/Seq2Seq/balanced_train.csv'
train_df = pd.read_csv(path_to_train)
path_to_val = '/content/drive/MyDrive/Seq2Seq/val.csv'
val_df = pd.read_csv(path_to_val)
# print(df.shape)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Create DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [8]:
model_name = "Helsinki-NLP/opus-mt-en-es"

# Loading tokenizer and model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [9]:
def preprocess_function(examples):
    # Tokenize inputs
    model_inputs = tokenizer(
        examples["english"],
        truncation=True,
        padding=True
    )

    # Tokenize targets
    labels = tokenizer(
        text_target=examples["spanish"],
        truncation=True,
        padding=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [10]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

Map:   0%|          | 0/113148 [00:00<?, ? examples/s]

Map:   0%|          | 0/8105 [00:00<?, ? examples/s]

In [11]:
def translate(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Generate output tokens
    translated_tokens = model.generate(**inputs)

    # Decode tokens to strng
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text


In [12]:
# examples = [
#     "hello, how are you?",
#     "i am learning deep learning",
#     "good night and sweet dreams",
#     "where is the library?",
#     "the cat is on the table",
#     "I am Pikachu"
# ]

# for sentence in examples:
#     print(f"{sentence} -> {translate(sentence)}")


In [13]:
import evaluate
bleu = evaluate.load("sacrebleu")


In [14]:
def generate_translation_batch(batch):
    inputs = tokenizer(batch["english"], return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(model.device)

    # Generate translations
    translated = model.generate(input_ids)

    # Decode
    batch["predictions"] = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return batch


In [15]:
# subset = tokenized_dataset["validation"].select(range(100))  # Testing on first 100 samples

# # Adding original texts back (not tokenized)
# subset = dataset["validation"].select(range(100)).map(generate_translation_batch, batched=True)


In [ ]:
subset = tokenized_dataset["validation"] 

# Adding original texts back 
subset = dataset["validation"].map(generate_translation_batch, batched=True)


Map:   0%|          | 0/8105 [00:00<?, ? examples/s]

In [17]:
# Prepare predictions and references
preds = subset["predictions"]
refs = [[ref] for ref in subset["spanish"]]  # sacrebleu needs nested list

# Computing BLEU
results = bleu.compute(predictions=preds, references=refs)
print(f"BLEU Score on validation subset without fine-tuning the model: {results['score']:.2f}")


BLEU Score on validation subset without fine-tuning the model: 56.74
